# Homework 6 - Recommendation System

In [199]:
from datetime import date
studentName="Qi Li"
studentEmail="li.qi8@northeastern.edu"
homework=6
contributedPercentage=100
print("IE 7275: Data mining in Engineering")
print("\n"*15)
print(f'Homework - {homework}')
print(f'Student name : {studentName}')
print(f'Student Email : {studentEmail}')
print("\n"*15)
print(f'Percentage of Effort Contributed by Student : {contributedPercentage}%')
print(f'Submission Date: {date.today()}')

IE 7275: Data mining in Engineering
















Homework - 6
Student name : Qi Li
Student Email : li.qi8@northeastern.edu
















Percentage of Effort Contributed by Student : 100%
Submission Date: 2023-04-25


### Import packages

In [1]:
!pip install mlxtend

In [2]:
!pip install surprise

In [100]:
import pandas as pd
import numpy as np
import heapq
import scipy
import dmba
import seaborn as sns
import random
import csv
import matplotlib.pyplot as plt
%matplotlib inline

from pathlib import Path
from collections import defaultdict
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Problem 1 - Entree Chicago Recommendation

Dataset - <a href="http://archive.ics.uci.edu/ml/datasets/Entree+Chicago+Recommendation+Data">Entree Chicago Recommendation</a>

Develop a recommendation engine for the city Chicago restaurants

What are the common association between those restraunts?

In [172]:
# load data
df1 = pd.read_csv('/Users/liqi/Desktop/IE7275/homework/hw6/chicago.csv', header = None, names = ['id','restuarant', 'feature'])
# drop the first column
df1 = df1.drop('id', axis=1)
df1

,restuarant,feature
0,Moti Mahal,214 035 149 021 117 075 204 051 163
1,Village,026 249 174 004 132 249 198 191 192 125 075 20...
2,Millrose Brewing Company,137 249 194 215 213 174 249 191 192 008 075 20...
3,Dover Straits,137 190 174 249 212 075 205 053 165
4,Eat Your Hearts Out,214 249 249 197 111 025 025 112 075 205 053 164
...,...,...
671,Dixie Que,214 249 194 215 213 174 100 086 249 063 059 23...
672,Star Top Cafe,214 068 076 205 053 166
673,Febo,214 174 198 191 192 125 075 205 052 164
674,Buster Crab Seafood,214 249 212 074 204 052 164


In [173]:
# split the feature column and convert the dataframe into long format
df1['feature'] = df1['feature'].str.split(' ')
df = df1.explode('feature')
# show the first 10 record
df.head(10)

,restuarant,feature
0,Moti Mahal,214
0,Moti Mahal,035
0,Moti Mahal,149
0,Moti Mahal,021
0,Moti Mahal,117
0,Moti Mahal,075
0,Moti Mahal,204
0,Moti Mahal,051
0,Moti Mahal,163
1,Village,026


In [174]:
#df_new = df1['feature'].str.split(expand=True)
#df = pd.concat([df1['name'], df_new], axis=1)
#df

In [175]:
# #Assume that for all features, they only appear one time
df['time'] = 1
df.head()

,restuarant,feature,time
0,Moti Mahal,214,1
0,Moti Mahal,035,1
0,Moti Mahal,149,1
0,Moti Mahal,021,1
0,Moti Mahal,117,1


In [176]:
# len(pd.unique(df.feature))

In [177]:
# check duplicate data
num_duplicates = df.duplicated().sum()
num_duplicates

99

In [178]:
# drop duplicate data
df = df.drop_duplicates()

In [180]:
# Converting data from long to wide format
df = df.pivot('restuarant', 'feature', 'time')
# Show the first 5 record
df.head()

feature,001,002,004,006,007,008,010,011,012,014,...,235,237,241,242,243,245,246,247,249,253
restuarant,,,,,,,,,,,,,,,,,,,,,
302 West,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94th Aero Squadron,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
AMBRIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
ARUN'S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
Abbis Abeba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
# Print the data for restuarant number 10
# df[df.index==3]
df.iloc[9:10,:]

feature,001,002,004,006,007,008,010,011,012,014,...,235,237,241,242,243,245,246,247,249,253
restuarant,,,,,,,,,,,,,,,,,,,,,
Alex's Washington Gardens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [190]:
# replave the NA value with 0
df.fillna(0, inplace = True)
# show the first 5 record
df.head()

feature,001,002,004,006,007,008,010,011,012,014,...,235,237,241,242,243,245,246,247,249,253
restuarant,,,,,,,,,,,,,,,,,,,,,
302 West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94th Aero Squadron,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
AMBRIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
ARUN'S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Abbis Abeba,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [191]:
# change the column name
feature_name = pd.read_csv('/Users/liqi/Desktop/chicago/entree/data/features.txt', header = None, names = ['id', 'feature'], delimiter='\t', dtype={'id': 'str'})
feature_map = dict(zip(feature_name['id'], feature_name['feature']))
df.columns = df.columns.map(feature_map)
df.head()

feature,Authentic,Afghanistan,After Hours Dining,American (New),American (Regional),American (Traditional),An Historic Spot,An Out Of The Way Find,Argentinean,Asian,...,Thai,Tourist Appeal,Ukranian,Up and Coming,Vegetarian,Very Busy - Reservations a Must,Vietnamese,Walk,Weekend Brunch,Wheelchair Access
restuarant,,,,,,,,,,,,,,,,,,,,,
302 West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94th Aero Squadron,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
AMBRIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
ARUN'S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Abbis Abeba,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Item-based Collaborative Filtering

In [192]:
df_cf = df.copy()
for i in range(0,len(df_cf.columns)) :
    # Loop through the columns for each column
    for j in range(0,len(df_cf.columns)) :
      # Fill in placeholder with cosine similarities
      df_cf.iloc[i,j] = 1-cosine(df_cf.iloc[:,i],df_cf.iloc[:,j])
df_cf.head()

feature,Authentic,Afghanistan,After Hours Dining,American (New),American (Regional),American (Traditional),An Historic Spot,An Out Of The Way Find,Argentinean,Asian,...,Thai,Tourist Appeal,Ukranian,Up and Coming,Vegetarian,Very Busy - Reservations a Must,Vietnamese,Walk,Weekend Brunch,Wheelchair Access
restuarant,,,,,,,,,,,,,,,,,,,,,
302 West,1.000000,0.0,0.132874,0.000000,0.000000,0.025516,6.154575e-02,0.031879,0.0,0.0,...,0.000000,0.064550,0.0,0.030096,0.0,0.026575,7.216878e-02,0.000000,0.169932,0.000000
94th Aero Squadron,0.000000,1.0,0.000000,0.000000,0.000000,0.088388,0.000000e+00,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.069252,0.000000
AMBRIA,0.136384,0.0,1.000000,0.076969,0.000000,0.097624,3.182188e-04,0.000086,0.0,0.0,...,0.000000,0.163591,0.0,0.038139,0.0,0.050474,4.374838e-04,0.030425,0.303908,0.000000
ARUN'S,0.000319,0.0,0.078445,1.000000,0.055724,0.055856,1.088584e-06,0.093243,0.0,0.0,...,0.030742,0.047136,0.0,0.217436,0.0,0.135583,1.754619e-06,0.034831,0.231445,0.033823
Abbis Abeba,0.000001,0.0,0.000213,0.058816,1.000000,0.000139,6.464066e-09,0.056174,0.0,0.0,...,0.000129,0.000293,0.0,0.104812,0.0,0.000354,1.221551e-08,0.000162,0.086860,0.000153


In [148]:
df_neighbours = pd.DataFrame(index = df.columns, columns = range(1,4))

In [150]:
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(df2.columns)):
    df_neighbours.iloc[i,:3] = df2.iloc[0:,i].sort_values(ascending=False)[:3].index
df3

,1,2,3
feature,,,
Authentic,302 West,Duke of Perth,Greek Islands
Afghanistan,94th Aero Squadron,Helmand,California Pizza Kitchen
After Hours Dining,El Presidente,Neon Greek Village,Village
American (New),Winnetka Grill,Vidalia on Park,Four Farthings Tavern & Grill
American (Regional),Zinfandel,Dish,Prairie
...,...,...,...
Very Busy - Reservations a Must,TOPOLOBAMPO,Geja's Cafe,TRIO
Vietnamese,Nhu Hoa Cafe,Da Luigino,Song Huong
Walk,Dionne's,Southport City Saloon,Fireplace Inn


In [94]:
# show the top 5 similar products of each product
df3 = pd.DataFrame(index = df2.columns, columns = range(1,6))
 
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(df2.columns)):
    df3.iloc[i,:6] = df2.iloc[0:,i].sort_values(ascending=False)[:5].index
    
display(df3.sample(5))

,1,2,3,4,5
feature,,,,,
Short Drive,La Creperie,Sabatino's,Robinson's No. 1 Ribs,Goose Island Brewing Co.,Golden Ox
American (New),Retreat,Lucky Platter,Vidalia on Park,Farrago,Crown Room
Indonesian,August Moon,302 West,P.J. Clarke's,Oodles of Noodles,Orbit
Scandinavian,Tre Kronor,302 West,Outback Steakhouse,Oo-La-La!,Oodles of Noodles
Coffeehouses,Mozart Cafe,302 West,P.S. Bangkok,Oodles of Noodles,Orbit


We can see from above table that if we would like to pick a 'Scandinavian' restaurant, the top 3 it will recommend are'Tre Kronor', '302 West', and 'Outback Steakhouse' restaurants.

### Association Rules

In [202]:
# create frequent itemsets
itemsets = apriori(df, min_support=0.2, use_colnames = True)
print(itemsets)

     support                                           itemsets
0   0.329882                                              (Cab)
1   0.318047                                       (Good Decor)
2   0.332840                                  (Excellent Decor)
3   0.522189                                   (Excellent Food)
4   0.215976                               (Extraordinary Food)
5   0.245562                                          (Italian)
6   0.281065                                       (Long Drive)
7   0.204142                                        (below $15)
8   0.233728                                          ($15-$30)
9   0.214497                                          ($15-$30)
10  0.485207                                    (Parking/Valet)
11  0.264793                                  (Private Parties)
12  0.264793                          (Private Rooms Available)
13  0.213018                                     (Good Service)
14  0.548817                            

In [200]:
# and convert into rules
rules = association_rules(itemsets, metric='confidence', min_threshold=0.5)
rules.sort_values(by=['lift'], ascending=False).head(6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
36,"(Parking/Valet, Private Parties)",(Private Rooms Available),0.204142,0.264793,0.204142,1.00000,3.776536,0.150087,inf,0.923792
18,(Private Rooms Available),(Private Parties),0.264793,0.264793,0.264793,1.00000,3.776536,0.194678,inf,1.000000
19,(Private Parties),(Private Rooms Available),0.264793,0.264793,0.264793,1.00000,3.776536,0.194678,inf,1.000000
35,"(Parking/Valet, Private Rooms Available)",(Private Parties),0.204142,0.264793,0.204142,1.00000,3.776536,0.150087,inf,0.923792
39,(Private Parties),"(Parking/Valet, Private Rooms Available)",0.264793,0.204142,0.204142,0.77095,3.776536,0.150087,3.4746,1.000000
38,(Private Rooms Available),"(Parking/Valet, Private Parties)",0.264793,0.204142,0.204142,0.77095,3.776536,0.150087,3.4746,1.000000


In [201]:
print(rules.sort_values(by=['lift'], ascending=False).drop(columns=['antecedent support', 'consequent support', 'conviction']).head(6))

                                 antecedents  \
36          (Parking/Valet, Private Parties)   
18                 (Private Rooms Available)   
19                         (Private Parties)   
35  (Parking/Valet, Private Rooms Available)   
39                         (Private Parties)   
38                 (Private Rooms Available)   

                                 consequents   support  confidence      lift  \
36                 (Private Rooms Available)  0.204142     1.00000  3.776536   
18                         (Private Parties)  0.264793     1.00000  3.776536   
19                 (Private Rooms Available)  0.264793     1.00000  3.776536   
35                         (Private Parties)  0.204142     1.00000  3.776536   
39  (Parking/Valet, Private Rooms Available)  0.204142     0.77095  3.776536   
38          (Parking/Valet, Private Parties)  0.204142     0.77095  3.776536   

    leverage  zhangs_metric  
36  0.150087       0.923792  
18  0.194678       1.000000  
19  0.194678

In [197]:
# filter to get rules with single consequents only
rules[[len(c) == 1 for c in rules.consequents]].sort_values(by=['lift'], ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
19,(Private Parties),(Private Rooms Available),0.264793,0.264793,0.264793,1.000000,3.776536,0.194678,inf,1.000000
36,"(Parking/Valet, Private Parties)",(Private Rooms Available),0.204142,0.264793,0.204142,1.000000,3.776536,0.150087,inf,0.923792
18,(Private Rooms Available),(Private Parties),0.264793,0.264793,0.264793,1.000000,3.776536,0.194678,inf,1.000000
35,"(Parking/Valet, Private Rooms Available)",(Private Parties),0.204142,0.264793,0.204142,1.000000,3.776536,0.150087,inf,0.923792
23,"(Excellent Service, Weekend Brunch)",(Excellent Decor),0.384615,0.332840,0.208580,0.542308,1.629333,0.080564,1.45766,0.627660


In [17]:
# Convert data set into a sparse data frame
sparse_df = df.astype(pd.SparseDtype(int, fill_value=0))
print('Density {}'.format(sparse_df.sparse.density))

# create frequent itemsets
itemsets = apriori(sparse_df, min_support=0.2, use_colnames=True)

# and convert into rules
rules = association_rules(itemsets, metric='confidence', min_threshold=0.5)
rules.sort_values(by=['lift'], ascending=False).head(6)

Density 0.06654013620631907


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
36,"(Parking/Valet, Private Parties)",(Private Rooms Available),0.204142,0.264793,0.204142,1.00000,3.776536,0.150087,inf,0.923792
18,(Private Rooms Available),(Private Parties),0.264793,0.264793,0.264793,1.00000,3.776536,0.194678,inf,1.000000
19,(Private Parties),(Private Rooms Available),0.264793,0.264793,0.264793,1.00000,3.776536,0.194678,inf,1.000000
35,"(Parking/Valet, Private Rooms Available)",(Private Parties),0.204142,0.264793,0.204142,1.00000,3.776536,0.150087,inf,0.923792
39,(Private Parties),"(Parking/Valet, Private Rooms Available)",0.264793,0.204142,0.204142,0.77095,3.776536,0.150087,3.4746,1.000000
38,(Private Rooms Available),"(Parking/Valet, Private Parties)",0.264793,0.204142,0.204142,0.77095,3.776536,0.150087,3.4746,1.000000


We can take a look at the confidence, if the confidence is closer than 1, then there is a stronger connection between the antecedents and the consequents. So we can see that the 'Private Parties' and the 'Private Rooms Available' as well as 'Private Parties' and 'Parking and Private Rooms Available' have a strong connection.

### Problem 2 - Animie Recommendation ##

Dataset - <a ref="https://www.kaggle.com/CooperUnion/anime-recommendations-database">Animie Recommendation</a>

Develop a animie recommendation engine based on the user reviews

What are the common association between those animies?

In [198]:
# load data
df21 = pd.read_csv('/Users/liqi/Desktop/IE7275/homework/hw6/anime.csv')
df22 = pd.read_csv('/Users/liqi/Desktop/IE7275/homework/hw6/rating.csv')
df21.head()
df22.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [20]:
df2 = pd.merge(df21, df22, on = 'anime_id')
# rename the columns
df2 = df2.rename(columns={
    'rating_x': 'rating', 
    'rating_y': 'user_rating'})
df2.head()

,anime_id,name,genre,type,episodes,rating,members,user_id,user_rating
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,99,5
1,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,152,10
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,244,10
3,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,271,10
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,278,-1


In [21]:
# remove the unnecessary rows such as -1 in user_rating
df2["user_rating"] = df2["user_rating"].replace({-1: np.nan})
# remove NA values
df2 = df2.dropna().iloc[:,1:]
# remove duplicate data
df = df.drop_duplicates()
df2.head()

,name,genre,type,episodes,rating,members,user_id,user_rating
0,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,99,5.0
1,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,152,10.0
2,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,244,10.0
3,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,271,10.0
5,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,322,10.0


In [62]:
data = df2.copy()

### Collaborative Filtering

In [72]:
df_sample = data.sample(frac = 0.1, random_state = 30)
df_sample

,name,genre,type,episodes,rating,members,user_id,user_rating
3392416,Shakugan no Shana II (Second),"Action, Drama, Fantasy, Romance, School, Super...",TV,24,7.79,184525,58972,6.0
4058367,"Yondemasu yo, Azazel-san. Z","Comedy, Demons, Supernatural",TV,13,7.65,20640,26215,8.0
4389741,Chobits,"Comedy, Drama, Ecchi, Romance, Sci-Fi, Seinen",TV,26,7.57,266846,32555,9.0
4911203,Shinrei Tantei Yakumo,"Horror, Mystery, Shoujo, Supernatural",TV,13,7.47,78952,46958,9.0
3074331,Romeo x Juliet,"Drama, Fantasy, Historical, Romance",TV,24,7.85,91530,64919,7.0
...,...,...,...,...,...,...,...,...
3816729,Tsubasa Chronicle 2nd Season,"Action, Adventure, Drama, Fantasy, Mystery, Ro...",TV,26,7.70,79166,40532,8.0
6551166,Princess Lover!,"Comedy, Ecchi, Harem, School",TV,12,7.03,104098,45263,9.0
6916987,Re-Kan!,"Comedy, School, Seinen, Supernatural",TV,13,6.83,37544,69850,3.0
7371411,Black Jack: Dr. Pinoko no Mori no Bouken,Comedy,Special,1,6.48,1272,40733,9.0


In [73]:
# Convert data set into the format required by the surprise package
# The columns must correspond to user id, item id and ratings (in that order)
reader = Reader(rating_scale=(1, 10))
data_ = Dataset.load_from_df(df_sample[['name', 'user_id', 'user_rating']], reader)

# Split into training and test set
trainset, testset = train_test_split(data_, test_size=.3, random_state=1)

In [74]:
# compute cosine similarity between users 
sim_options = {'name': 'cosine', 'user_based': True}
algo = KNNBasic(sim_options = sim_options)
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [75]:
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    byUser = defaultdict(list)
    for p in predictions:
        byUser[p.uid].append(p)
    
    # For each user, reduce predictions to top-n
    for uid, userPredictions in byUser.items():
        byUser[uid] = heapq.nlargest(n, userPredictions, key=lambda p: p.est)
    return byUser

In [76]:
# predict ratings for all pairs (u, i) that are NOT in the training set
predictions = algo.test(testset)
top_n = get_top_n(predictions, n = 4)

In [77]:
# Print the recommended items for each user
print('Top-4 recommended animes for each user')
for uid, user_ratings in list(top_n.items())[:5]:
    print('Anime {}'.format(uid))
    for prediction in user_ratings:
        print('  User {0.iid} ({0.est:.2f})'.format(prediction), end='')
    print()

Top-4 recommended animes for each user
Anime Memories
  User 46210 (9.71)  User 11134 (9.50)  User 34292 (9.00)  User 3979 (8.80)
Anime Tengen Toppa Gurren Lagann
  User 52168 (10.00)  User 65767 (10.00)  User 23691 (10.00)  User 8473 (10.00)
Anime Darker than Black: Kuro no Keiyakusha Gaiden
  User 19150 (10.00)  User 15127 (9.77)  User 13266 (9.50)  User 32180 (9.36)
Anime Needless
  User 45314 (10.00)  User 30322 (9.60)  User 55369 (9.45)  User 67472 (9.43)
Anime Nabari no Ou
  User 69737 (10.00)  User 4910 (10.00)  User 65948 (9.90)  User 66101 (9.75)


In the example above, we can see that the 'Anime Memories' will be recommended to user 46210, user 11134, User 34292 and User 3979; and the 'Anime Nabari no Ou' will be recommended to User 69737, User 4910, User 65948, and User 66101.

### Association Rules

In [85]:
# create frequent itemsets
# choose the anime with over 25000 ratings
data = df2.copy()
data['time'] = 1
counts = data['user_id'].value_counts()
counts2 = data['name'].value_counts()
data = data[data['name'].isin(counts2[counts2 >= 25000].index)]
data = data.sample(frac = 0.1, random_state = 42)
data = data.pivot_table(index = 'user_id', columns = 'name', values = 'time')
# Replace NA with 0 
data.fillna(0, inplace=True)

In [86]:
itemsets = apriori(data, min_support=0.02, use_colnames=True)
print(itemsets)

    support                                itemsets
0  0.424938                            (Death Note)
1  0.324314                    (Shingeki no Kyojin)
2  0.320948                      (Sword Art Online)
3  0.023317        (Death Note, Shingeki no Kyojin)
4  0.023691          (Death Note, Sword Art Online)
5  0.024938  (Shingeki no Kyojin, Sword Art Online)


In [87]:
# convert into rules
rules = association_rules(itemsets, metric='confidence', min_threshold=0.02)
rules.sort_values(by=['lift'], ascending=False).head(6)

print(rules.sort_values(by=['lift'], ascending=False)
      .drop(columns=['antecedent support', 'consequent support', 'conviction'])
      .head(6))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4,(Shingeki no Kyojin),(Sword Art Online),0.324314,0.320948,0.024938,0.076894,0.239583,-0.079150,0.735616,-0.824479
5,(Sword Art Online),(Shingeki no Kyojin),0.320948,0.324314,0.024938,0.077700,0.239583,-0.079150,0.732610,-0.823759
3,(Sword Art Online),(Death Note),0.320948,0.424938,0.023691,0.073815,0.173708,-0.112692,0.620894,-0.875079
2,(Death Note),(Sword Art Online),0.424938,0.320948,0.023691,0.055751,0.173708,-0.112692,0.719146,-0.892146
1,(Shingeki no Kyojin),(Death Note),0.324314,0.424938,0.023317,0.071895,0.169191,-0.114497,0.619609,-0.879043
0,(Death Note),(Shingeki no Kyojin),0.424938,0.324314,0.023317,0.054871,0.169191,-0.114497,0.714914,-0.895168


            antecedents           consequents   support  confidence      lift  \
4  (Shingeki no Kyojin)    (Sword Art Online)  0.024938    0.076894  0.239583   
5    (Sword Art Online)  (Shingeki no Kyojin)  0.024938    0.077700  0.239583   
3    (Sword Art Online)          (Death Note)  0.023691    0.073815  0.173708   
2          (Death Note)    (Sword Art Online)  0.023691    0.055751  0.173708   
1  (Shingeki no Kyojin)          (Death Note)  0.023317    0.071895  0.169191   
0          (Death Note)  (Shingeki no Kyojin)  0.023317    0.054871  0.169191   

   leverage  zhangs_metric  
4 -0.079150      -0.824479  
5 -0.079150      -0.823759  
3 -0.112692      -0.875079  
2 -0.112692      -0.892146  
1 -0.114497      -0.879043  
0 -0.114497      -0.895168  


In [88]:
# filter to get rules with single consequents only
rules[[len(c) == 1 for c in rules.consequents]].sort_values(by=['lift'], ascending=False).head(6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4,(Shingeki no Kyojin),(Sword Art Online),0.324314,0.320948,0.024938,0.076894,0.239583,-0.079150,0.735616,-0.824479
5,(Sword Art Online),(Shingeki no Kyojin),0.320948,0.324314,0.024938,0.077700,0.239583,-0.079150,0.732610,-0.823759
3,(Sword Art Online),(Death Note),0.320948,0.424938,0.023691,0.073815,0.173708,-0.112692,0.620894,-0.875079
2,(Death Note),(Sword Art Online),0.424938,0.320948,0.023691,0.055751,0.173708,-0.112692,0.719146,-0.892146
1,(Shingeki no Kyojin),(Death Note),0.324314,0.424938,0.023317,0.071895,0.169191,-0.114497,0.619609,-0.879043
0,(Death Note),(Shingeki no Kyojin),0.424938,0.324314,0.023317,0.054871,0.169191,-0.114497,0.714914,-0.895168


We can see from above table that the antecedents and consequents does not have a strong consequese which is greater than 0.7. However, Sword Art Online and Death Note as well as Shingeki no Kyojin and Death Note both have consequent support scores at about 0.4, so there seems to have some relationship between these animes.